## 기업 고유코드
+ 작성: 임경호
+ 금융감독원 OPEN DART https://opendart.fss.or.kr/

In [1]:
import requests

url = 'https://opendart.fss.or.kr/api/corpCode.xml'
api_key = 'f2e08d4ed3de0ba3d5cbf59c04c223e02b1751a2'
params = {
    'crtfc_key': api_key,
}

response = requests.get(url, params=params)
if response.status_code != 200:     # URL GET '200 정상'
    print("URL GET Error")

In [2]:
import io
import zipfile

# 출력포멧이 Zip FILE (binary)
f = io.BytesIO(response.content)
zfile = zipfile.ZipFile(f)

In [3]:
zfile.namelist()

['CORPCODE.xml']

In [4]:
import xmltodict
# 압축된 파일에서 데이터를 읽습니다. xmltodict 모듈을 사용해서 압축이 해제된 xml 데이터를 파이썬의 OrderedDict 타입으로 변환합니다.
xml_data = zfile.read("CORPCODE.xml").decode("utf-8")
dict_data = xmltodict.parse(xml_data)

In [5]:
# dict_data

In [5]:
import pandas as pd
# 딕셔너리에서 키 값을 확인하여 실제 데이터가 있는 부분을 인덱싱합니다. 그러다음 해당 데이터를 판다스의 데이터프레임 객체로 변환합니다.
data_list = dict_data['result']['list']
df_corp = pd.DataFrame(data_list)

In [7]:
# df_corp.dtypes
# df_corp.head()

In [8]:
# corp_name 문자열 최대 길이 자리수
# df_corp['corp_name'].str.len().max()

In [6]:
corp_name = ['대성에너지', '삼천리', '경동도시가스', '서울도시가스', '인천도시가스', '부산도시가스', '씨엔씨티에너지']
df_corp_save = df_corp.query('corp_name in @corp_name').sort_values('corp_name')
df_corp_save

,corp_code,corp_name,stock_code,modify_date
81059,01208885,경동도시가스,267290,20230215
78531,00787376,대성에너지,117580,20230213
63127,00123967,부산도시가스,015350,20220217
85236,00128555,삼천리,004690,20230106
76534,00130684,서울도시가스,017390,20221205
82613,00152011,씨엔씨티에너지,None,20221214
98132,00145862,인천도시가스,034590,20230320


In [7]:
df_corp_save['stock_code'].fillna('', inplace=True)
df_corp_save

,corp_code,corp_name,stock_code,modify_date
81059,01208885,경동도시가스,267290,20230215
78531,00787376,대성에너지,117580,20230213
63127,00123967,부산도시가스,015350,20220217
85236,00128555,삼천리,004690,20230106
76534,00130684,서울도시가스,017390,20221205
82613,00152011,씨엔씨티에너지,,20221214
98132,00145862,인천도시가스,034590,20230320


### 코드 리스트를 DB Table에 저장

In [20]:
import sys
module_path = "D:\PythonProject\data-gatherer\common"
sys.path.append(module_path)
import dbconnect

conn = dbconnect.db_connect("DEMO_DW")
cur = conn.cursor()

for row in df_corp_save.itertuples():
    sql = "insert into dart_co_code(corp_code, corp_name, stock_code, modify_date) values (%s, %s, %s, %s)"
    cur.execute(sql, (row[1], row[2], row[3], row[4]))

conn.commit()
conn.close()

In [ ]:
df_corp.query('corp_name in @corp_name').sort_values('corp_name')['corp_code'].to_list()

['01208885',
 '00787376',
 '00123967',
 '00128555',
 '00130684',
 '00152011',
 '00145862']

In [ ]:
# ticker (stock_code)
df_corp.query('corp_name in @corp_name').sort_values('corp_name')['stock_code'].to_list()

['267290', '117580', '015350', '004690', '017390', None, '034590']

In [ ]:
# 회사명에 특정 문자열을 포함한 목록 찾기
df_corp[df_corp['corp_name'].str.contains('씨엔씨티')]

,corp_code,corp_name,stock_code,modify_date
82676,00152011,씨엔씨티에너지,None,20221214
